# View LSSTComCam DeepCoadd

- author Sylvie Dagoret-Campagne
- creation date 2025-04-30
- last update 2025-04-30
- LSST pipelines : w_2025_10

## Import

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
#import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler

import lsst.geom as geom
from lsst.geom import SpherePoint, degrees
import lsst.afw.display as afwDisplay

## Config

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-40356":
repo = '/repo/main'
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_05/DM-48666' # work
#collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810' # work
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359' # work


# bad : crash collection = 'LSSTComCam/runs/DRP/DP1/w_2025_08/DM-49029'

# bad : collection = "LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233"

# not working perhaps because I am using w_2025_10 version
# bad : no ccd visit collection = "LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864"
# bad : no ccd visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_15/DM-50050'
# bad : no cce visit collection = 'LSSTComCam/runs/DRP/DP1/w_2025_14/DM-49864'
# bad : no cce visit collection collection = 'LSSTComCam/runs/DRP/DP1/w_2025_13/DM-49751'
instrument = "LSSTComCam"
skymapName = "lsst_cells_v1"
where_clause = "instrument = \'" + instrument+ "\'"
collectionStr = collection.replace("/", "_")
BANDSEL = "r" # Most fields were observed in red filter

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection)
registry = butler.registry

In [ ]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collection)

## List of Sky field of interest

In [ ]:
lsstcomcam_targets = {}
lsstcomcam_targets["47 Tuc"] = {"field_name" : "47 Tuc Globular Cluster","ra"  : 6.02,"dec" : -72.08}
lsstcomcam_targets["Rubin SV 38 7"] = {"field_name" : "Low Ecliptic Latitude Field", "ra"  : 37.86,"dec" : 6.98}
lsstcomcam_targets["Fornax dSph"] = {"field_name" : "Fornax Dwarf Spheroidal Galaxy", "ra"  :40.0 ,"dec" : -34.45}
lsstcomcam_targets["ECDFS"] = {"field_name" : "Extended Chandra Deep Field South", "ra"  : 53.13 ,"dec" : -28.10 }
lsstcomcam_targets["EDFS"] = {"field_name" : "Euclid Deep Field South", "ra"  : 59.10 ,"dec" :-48.73 }
lsstcomcam_targets["Rubin SV 95 -25"] = {"field_name" : "Low Galactic Latitude Field", "ra" : 95.00 ,"dec" :-25.0 }
lsstcomcam_targets["Seagull"] = {"field_name" : "Seagull Nebula", "ra"  : 106.23,"dec" : -10.51 }

### Select the target

In [ ]:
#the_target = lsstcomcam_targets["Seagull"]
#the_target = lsstcomcam_targets["47 Tuc"] # bad
#the_target = lsstcomcam_targets["Fornax dSph"]
#the_target = lsstcomcam_targets["ECDFS"]

#key = "Seagull"
#key = "Fornax dSph"
#key = "ECDFS"
key = "EDFS"
the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_title = the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
target_point = SpherePoint(target_ra, target_dec, degrees)

## Find the list of tract numbers from Object Table

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype,  collections=collection)
tractsId_list = np.unique([ref.dataId['tract'] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Search all deepCoadd

- deepCoadd_calexp comes with WCS

In [ ]:
# List all  deepCoadd_calexp which are in the butler collection
# Thus all patch and tracts
#refs = butler.registry.queryDatasets("deepCoadd_calexp", collections = collection)
#for ref in refs:
#    print(ref.dataId)

## Find the DataId

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
print("Patch Index :", patch_info.getIndex()," , ",patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

In [ ]:
central_patch = patch_info.getIndex()
central_x, central_y = patch_info.getIndex()
neighbor_patches = [
    f"{x},{y}"
    for x in range(central_x - 1, central_x + 2)
    for y in range(central_y - 1, central_y + 2)
    if 0 <= x <= 8 and 0 <= y <= 8
]

In [ ]:
neighbor_patches

In [ ]:
# Add the patch and band to the dataId, we didn't need them for the objectTable_tract because it covers all patches and bands
# However the coadds are stored by patch and band dimensions so we have to add them to the dataId

dataId = {
    "band": BANDSEL,
    "tract": tract_info.getId(),
    "patch": patch_info.getSequentialIndex(),
    "skymap": 'lsst_cells_v1'
}


## Fetch the DeepCoadd

In [ ]:
coadd_exp = butler.get("deepCoadd_calexp", dataId)

## Plot in matplotlib

In [ ]:
image_array = coadd_exp.image.array
image = coadd_exp.image
wcs = coadd_exp.getWcs()
psf = coadd_exp.getPsf()

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(10,10))
im = ax.imshow(image_array, cmap="gray", origin="lower", vmin=0, vmax=2000)
ax.set_title(target_title)
plt.colorbar(im, ax=ax)
plt.show()


In [ ]:
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval

# Get astropy WCS to plot accordingly
wcs_astropy = WCS(wcs.getFitsMetadata())  # Alternative en extrayant l'entête FITS

# Use zscale to norm
interval = ZScaleInterval()
vmin, vmax = interval.get_limits(image_array)


fig  = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, projection=wcs_astropy)
im = ax.imshow(image_array, origin="lower", cmap="gray", vmin=vmin, vmax=vmax)

ax.set_xlabel("RA (deg)")
ax.set_ylabel("Dec (deg)")
ax.coords.grid(True, color="white", ls="dotted")
plt.title("DeepCoadd_calexp for " + target_title)
#plt.colorbar(im, ax=ax)
plt.show()


## Plot with Firefly

In [ ]:
afwDisplay.setDefaultBackend("firefly")
#display = afwDisplay.Display(frame=1)
#display.scale("asinh", "zscale")
#display.mtv(image, title = target_title)

In [ ]:
bands = ["u","g","r","i","z","y"]
all_dataIds = [  {
    "band": band,
    "tract": tract_info.getId(),
    "patch": patch_info.getSequentialIndex(),
    "skymap": 'lsst_cells_v1'
} for band in bands ]

In [ ]:
all_dataIds

In [ ]:
all_deepCoadds = []
all_titles = []
for idx,band  in enumerate(bands):
    the_band = bands[idx]
    the_dataId = all_dataIds[idx]
    the_title = key + f" band {the_band}" 
    try:
        coadd_exp = butler.get("deepCoadd_calexp", the_dataId)
        all_deepCoadds.append(coadd_exp )
        all_titles.append(the_title)

    except Exception as inst:
        print(f"{key} :: catch Exception for band {band}")
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str_

In [ ]:
N = len(all_deepCoadds)
for count in range(N):
    display = afwDisplay.Display(frame=count+1)
    # cannot succeed to show white stars on dark sky
    display.setImageColormap('gray')
    display.scale("asinh", "zscale")
    display.mtv(all_deepCoadds[count].image, title=all_titles[count])

In [ ]:
#display.clearViewer()

In [ ]:
#setImageColormap) are “gray” and “grey”